In [18]:
#CS5228 project

# data manipulation
import pandas as pd
import numpy as np
import math
import random
import pickle
import torch.optim as optim


# visualiation
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

# model training
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# classifiers
from sklearn.naive_bayes import GaussianNB # naive bayes
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.svm import SVC # SVM
from sklearn.linear_model import LogisticRegression # logistic regression
from sklearn.tree import DecisionTreeClassifier # decision Tree
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFE # for feature selection of LR
from sklearn.ensemble import BaggingClassifier 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier


# ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [19]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        #self.conv1 = nn.Conv2d(1, 6, 5)
        #self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(64, 100)
        self.fc2 = nn.Linear(100, 150)
        self.fc3 = nn.Linear(150, 200)
        self.fc4 = nn.Linear(200, 100)
        self.fc5 = nn.Linear(100, 50)
        self.fc6 = nn.Linear(50, 1)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        #x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        #x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        #x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.sigmoid(self.fc6(x))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [28]:
#load into dataframe
f_data = pd.read_csv('financial_data.csv', na_values=['?']) 
revealed = pd.read_csv('augmented_revealed_businesses_01.csv')
t_data = pd.read_csv('testing_data.csv', na_values=['?'])

display(f_data.head())
display(revealed.head())

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var56,Var57,Var58,Var59,Var60,Var61,Var62,Var63,Var64,Var65
0,18399,0.023954,0.150120,0.395670,3.63570,54.043,0.028822,0.031029,4.56831,1.01120,...,3871.001,0.011041,0.034914,0.98896,0.000000,9.5214,5.8248,34.713,10.5150,3.4752
1,15092,0.049699,0.065808,0.726800,12.94400,233.110,0.000000,0.063192,14.19601,0.89618,...,8751.901,0.059565,0.053189,0.93169,0.000000,5.0492,11.1520,24.784,14.7270,4.2204
2,19821,-0.356310,0.392880,0.158840,1.40430,-2.619,-0.085597,-0.356320,1.54531,0.92963,...,44.859,-0.172770,-0.586910,1.38330,0.000000,5.6112,15.7790,154.260,2.3662,2.0738
3,14171,0.001417,0.708110,-0.052312,0.88978,-31.198,0.269520,0.001407,0.41222,1.96540,...,-331.879,-0.000535,0.004820,0.99930,0.745480,17.1011,7.9482,88.147,4.1408,3.4021
4,12900,0.020041,0.346520,0.335930,2.76130,39.050,0.000000,0.020031,1.88591,1.29750,...,38170.001,0.212410,0.030652,0.80158,0.000862,9.7670,6.7570,53.651,6.8032,2.7412


,Var1,Var66
0,18399,0
1,19821,0
2,17769,0
3,19309,0
4,20728,0


In [21]:
#f_data_zeroes = f_data.replace(0, np.nan)
def fill_empty_std(dataframe):
    ones = revealed[revealed.Var66==1].Var1
    ones_df = f_data_zeros[f_data_zeros.Var1.isin(ones)]
    #display(ones_df)
    zeroes = revealed[revealed.Var66==0].Var1
    zeroes_df = f_data_zeros[f_data_zeros.Var1.isin(zeroes)]
    #display(zeroes_df)
    for index, row in dataframe.iterrows():
        for column in dataframe:
            #display((revealed[revealed.Var1==row.Var1].Var66==1).bool())
            if ((revealed[revealed.Var1==row.Var1].Var66==1).bool):
                col = ones_df[column]
            else:
                col = zeroes_df[column]
            mean = col.mean()
            standard_deviation = col.std()
            if math.isnan(row[column]):
                replacement_value = np.random.normal(mean, standard_deviation)
                dataframe.at[index, column] = replacement_value
                #print("Replaced empty value with {}".format(replacement_value))

In [11]:
f_data.dtypes

Var1       int64
Var2     float64
Var3     float64
Var4     float64
Var5     float64
Var6     float64
Var7     float64
Var8     float64
Var9     float64
Var10    float64
Var11    float64
Var12    float64
Var13    float64
Var14    float64
Var15    float64
Var16    float64
Var17    float64
Var18    float64
Var19    float64
Var20    float64
Var21    float64
Var22    float64
Var23    float64
Var24    float64
Var25    float64
Var26    float64
Var27    float64
Var28    float64
Var29    float64
Var30    float64
          ...   
Var36    float64
Var37    float64
Var38    float64
Var39    float64
Var40    float64
Var41    float64
Var42    float64
Var43    float64
Var44    float64
Var45    float64
Var46    float64
Var47    float64
Var48    float64
Var49    float64
Var50    float64
Var51    float64
Var52    float64
Var53    float64
Var54    float64
Var55    float64
Var56    float64
Var57    float64
Var58    float64
Var59    float64
Var60    float64
Var61    float64
Var62    float64
Var63    float

In [25]:
#clean data
clean_and_save = True
f_data_zeros = f_data.replace(0, np.nan)
                
if clean_and_save:
    f_data_clean = f_data.replace(0, np.nan) # assume 0 values also means missing
    fill_empty_std(f_data_clean)
    f_data_clean.to_pickle("./f_data_clean.pkl")

t_data_clean = t_data.replace(0, np.nan)
fill_empty_std(t_data_clean)


In [101]:
f_data_clean.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var56,Var57,Var58,Var59,Var60,Var61,Var62,Var63,Var64,Var65
0,18399,0.023954,0.150120,0.395670,3.63570,54.043,0.028822,0.031029,4.56831,1.01120,...,3871.001,0.011041,0.034914,0.98896,24.619609,9.5214,5.8248,34.713,10.5150,3.4752
1,15092,0.049699,0.065808,0.726800,12.94400,233.110,-0.265134,0.063192,14.19601,0.89618,...,8751.901,0.059565,0.053189,0.93169,-27.933393,5.0492,11.1520,24.784,14.7270,4.2204
2,19821,-0.356310,0.392880,0.158840,1.40430,-2.619,-0.085597,-0.356320,1.54531,0.92963,...,44.859,-0.172770,-0.586910,1.38330,38.349935,5.6112,15.7790,154.260,2.3662,2.0738
3,14171,0.001417,0.708110,-0.052312,0.88978,-31.198,0.269520,0.001407,0.41222,1.96540,...,-331.879,-0.000535,0.004820,0.99930,0.745480,17.1011,7.9482,88.147,4.1408,3.4021
4,12900,0.020041,0.346520,0.335930,2.76130,39.050,0.525878,0.020031,1.88591,1.29750,...,38170.001,0.212410,0.030652,0.80158,0.000862,9.7670,6.7570,53.651,6.8032,2.7412


In [50]:
#preprocess data
fdatan = f_data_clean.copy(deep=True)
normald = StandardScaler()
fdatan_1 = normald.fit_transform(fdatan.ix[:,fdatan.columns!="Var1"]) # normalize all variables except Var1
fdatan = pd.DataFrame(np.column_stack((fdatan["Var1"].values,fdatan_1)),
                      columns = fdatan.columns).set_index(fdatan.index)
tdatan = t_data_clean.copy(deep=True)
normald = StandardScaler()
tdatan_1 = normald.fit_transform(tdatan.ix[:,tdatan.columns!="Var1"]) # normalize all variables except Var1
tdatan = pd.DataFrame(np.column_stack((tdatan["Var1"].values,tdatan_1)),
                      columns = tdatan.columns).set_index(tdatan.index)

fdatan.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var56,Var57,Var58,Var59,Var60,Var61,Var62,Var63,Var64,Var65
0,18399.0,-0.048725,-0.085706,0.051995,-0.012599,0.012602,0.034632,-0.057168,-0.019404,-0.104331,...,-0.048021,0.010508,0.003152,-0.017767,0.097160,-0.015544,-0.099273,-0.002151,0.041826,-0.078030
1,15092.0,-0.009047,-0.102239,0.117008,0.003875,0.014002,-0.136795,-0.007851,-0.005824,-0.118432,...,0.044772,0.011312,0.004065,-0.018531,0.402026,-0.015659,-0.037866,-0.002508,0.179037,-0.076358
2,19821.0,-0.634787,-0.038104,0.005497,-0.016549,0.012159,0.013889,-0.651114,-0.023668,-0.114331,...,-0.120762,0.007462,-0.027887,-0.012511,0.709256,-0.015645,0.015471,0.002154,-0.223630,-0.081173
3,14171.0,-0.083459,0.023709,-0.035960,-0.017459,0.011935,0.078266,-0.102590,-0.025266,0.012649,...,-0.127924,0.010316,0.001650,-0.017629,-0.028316,-0.015350,-0.074796,-0.000227,-0.165821,-0.078194
4,12900.0,-0.054756,-0.047194,0.040266,-0.014147,0.012485,0.219900,-0.074032,-0.023188,-0.069232,...,0.604054,0.013846,0.002940,-0.020265,-0.036850,-0.015538,-0.088528,-0.001469,-0.079090,-0.079676


In [51]:
#merge dataframe
fdatan_merged = fdatan.merge(revealed, how="outer")
fdatan_train1 = fdatan.merge(revealed) # dataframe, whose bankruptcy status is known (0 = good standing, 1 = bankrupt)

display(revealed.head())
display(len(fdatan_train1[fdatan_train1.Var66==0].index))
len(fdatan_train1[fdatan_train1.Var66==1].index)

,Var1,Var66
0,18399,0
1,19821,0
2,17769,0
3,19309,0
4,20728,0


8832

171

In [52]:
#classifier here
x1 = fdatan_train1.drop(columns=['Var1', 'Var66']) # Var 1 is company ID, Var 66 is the status
y1 = fdatan_train1['Var66']
t1 = tdatan.drop(columns=['Var1'])
cID = t_data['Var1'].tolist() # use original values, since nothing is done in cID of tdata
clas = AdaBoostClassifier() # select classifier here
clas.fit(x1, y1)

# write results to predictions.csv
f = open('predictions_std_dev.csv', 'w')
f.write('Business_ID,Is_Bankrupted\n')
for a,b in zip(cID, clas.predict(t1)):
    f.write(str(a))
    f.write(',')
    f.write(str(round(b)))
    f.write('\n')
f.close()

In [53]:
index_to_round = round(len(fdatan_train1.index)*0.75)
display("Dataset will be split on index: {}".format(index_to_round))

for_training = fdatan_train1.iloc[:index_to_round, :]
for_testing = fdatan_train1.iloc[index_to_round:, :]

f_data_positives = for_training[for_training.Var66==0]
f_data_negatives = for_training[for_training.Var66==1]

x_pos = f_data_positives.drop(columns=['Var1', 'Var66'])
y_pos = f_data_positives.Var66
x_neg = f_data_negatives.drop(columns=['Var1', 'Var66'])
y_neg = f_data_negatives.Var66

x_testing = for_testing.drop(columns=['Var1', 'Var66'])
y_testing = for_testing.Var66

display(len(x_pos.index))
display(len(x_neg.index))

'Dataset will be split on index: 6752'

6624

128

In [55]:
x_train1 = for_training.drop(columns=['Var1', 'Var66'])
y_train1 = for_training.Var66

x_test1 = for_testing.drop(columns=['Var1', 'Var66'])
y_test1 = for_testing.Var66

clf = GradientBoostingClassifier(learning_rate=0.1)
clf.fit(x_train1, y_train1)
train_score = clf.score(x_train1, y_train1)
test_score = clf.score(x_test1, y_test1)
test_f1 = f1_score(y_test1, clf.predict(x_test1))
print('Train Score:{} Test Score:{}'.format(train_score, test_score))
print('Test F1:{}'.format(test_f1))

Train Score:0.9948163507109005 Test Score:0.9773434029320303
Test F1:0.2608695652173913


In [56]:
# write results to predictions.csv
f = open('predictions_gradient_boost.csv', 'w')
f.write('Business_ID,Is_Bankrupted\n')
for a,b in zip(cID, clf.predict(t1)):
    f.write(str(a))
    f.write(',')
    f.write(str(round(b)))
    f.write('\n')
f.close()

In [57]:
net = Net()
print(net)

Net(
  (fc1): Linear(in_features=64, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=150, bias=True)
  (fc3): Linear(in_features=150, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=100, bias=True)
  (fc5): Linear(in_features=100, out_features=50, bias=True)
  (fc6): Linear(in_features=50, out_features=1, bias=True)
)


In [58]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


cpu


In [64]:
for epoch in range(200):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(0, len(x_neg.index)):
        x_train = torch.Tensor(x_neg.iloc[i].values).float()
        y_train = torch.tensor(1).float()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(x_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
        x_train = torch.Tensor(x_pos.iloc[np.random.randint(0, len(x_pos.index))].values).float()
        y_train = torch.tensor(0).float()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(x_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i == len(x_neg.index)-1:    # print every last mini-batch
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,   128] loss: 0.015
[2,   128] loss: 0.008
[3,   128] loss: 0.008
[4,   128] loss: 0.008
[5,   128] loss: 0.009
[6,   128] loss: 0.014
[7,   128] loss: 0.012
[8,   128] loss: 0.009
[9,   128] loss: 0.010
[10,   128] loss: 0.006
[11,   128] loss: 0.024
[12,   128] loss: 0.009
[13,   128] loss: 0.004
[14,   128] loss: 0.012
[15,   128] loss: 0.008
[16,   128] loss: 0.009
[17,   128] loss: 0.010
[18,   128] loss: 0.010
[19,   128] loss: 0.008
[20,   128] loss: 0.009
[21,   128] loss: 0.011
[22,   128] loss: 0.006
[23,   128] loss: 0.003
[24,   128] loss: 0.014
[25,   128] loss: 0.006
[26,   128] loss: 0.007
[27,   128] loss: 0.012
[28,   128] loss: 0.015
[29,   128] loss: 0.010
[30,   128] loss: 0.010
[31,   128] loss: 0.014
[32,   128] loss: 0.006
[33,   128] loss: 0.008
[34,   128] loss: 0.009
[35,   128] loss: 0.008
[36,   128] loss: 0.003
[37,   128] loss: 0.004
[38,   128] loss: 0.004
[39,   128] loss: 0.011
[40,   128] loss: 0.009
[41,   128] loss: 0.008
[42,   128] loss: 0.007
[

In [65]:
correct = 0
wrong = 0
for i in range(0, len(x_testing.index)):
    ref_x_test1 = torch.tensor(x_testing.iloc[i].values).float()
    ref_y_test1 = torch.tensor(y_testing.iloc[i]).float()
    outputs = net(ref_x_test1)
    if (round(ref_y_test1.item()) == round(outputs.item())):
        correct += 1
    else:
        wrong += 1
    #print('GroundTruth: {}'.format(ref_y_test1.item()))
    #print('Predicted: {}'.format(outputs.item()))
display('Number of correct: {}'.format(correct))
display('Number of wrong: {}'.format(wrong))

'Number of correct: 2140'

'Number of wrong: 111'

In [46]:
print(t1)

          Var2      Var3      Var4      Var5      Var6      Var7      Var8  \
0     0.111465 -0.059198  0.082161  0.070111  0.214412  0.025733  0.090308   
1     0.312120  0.003717  0.016274 -0.129643  0.017681 -0.046425  0.351322   
2    -0.040492 -0.054128  0.046884 -0.059626  0.048388  0.025733 -0.060728   
3    -0.052139 -0.019013 -0.006665 -0.158121 -0.397623  0.025481 -0.072305   
4    -0.006261 -0.040289  0.045632 -0.093610  0.032537  0.025733 -0.026704   
5    -0.009200 -0.020181  0.014879 -0.120433  0.010136  0.041498 -0.021918   
6     0.004889 -0.024099  0.010871 -0.131832  0.005827  0.046135 -0.003728   
7    -0.055094 -0.065724  0.040546  0.053787  0.089897  0.025831 -0.062399   
8     0.015653 -0.006586  0.028436 -0.120079  0.007174  0.012307 -0.007610   
9     0.272167 -0.018216  0.006892 -0.135244  0.015968  0.018217  0.327876   
10    0.183961 -0.021378  0.044627 -0.108459  0.030473  0.025733  0.229118   
11   -0.032534 -0.019226  0.026292 -0.118628  0.020075  0.025733

In [66]:
f = open('predictions_nn_200_aug.csv', 'w')
f.write('Business_ID,Is_Bankrupted\n')
for i in range(0, len(t1.index)):
#     item = torch.Tensor(t1.iloc[i].values)
#     print(type(item))
#     print(item)
#     print(item.float())
    x_test = torch.Tensor(t1.iloc[i].values).float()
#     if item:
#         x_test = torch.Tensor(t1.iloc[i].values).float()
#     else:
#         print('no item?')
    outputs = net(x_test)
    print('output is: ' + str(outputs))
    f.write(str(cID[i]))
    f.write(',')
    f.write(str(round(outputs.item())))
    f.write('\n')
f.close()

output is: tensor([1.1527e-08], grad_fn=<SigmoidBackward>)
output is: tensor([0.0121], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([4.7811e-13], grad_fn=<SigmoidBackward>)
output is: tensor([0.0110], grad_fn=<SigmoidBackward>)
output is: tensor([1.6977e-05], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([4.4513e-23], grad_fn=<SigmoidBackward>)
output is: tensor([4.8009e-07], grad_fn=<SigmoidBackward>)
output is: tensor([1.5823e-08], grad_fn=<SigmoidBackward>)
output is: tensor([1.0382e-05], grad_fn=<SigmoidBackward>)
output is: tensor([0.6642], grad_fn=<SigmoidBackward>)
output is: tensor([2.0163e-19], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([0.0002], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([0.1038], grad_fn=<SigmoidBackward>

output is: tensor([4.7664e-18], grad_fn=<SigmoidBackward>)
output is: tensor([0.0027], grad_fn=<SigmoidBackward>)
output is: tensor([0.9998], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([1.4353e-22], grad_fn=<SigmoidBackward>)
output is: tensor([4.6445e-16], grad_fn=<SigmoidBackward>)
output is: tensor([0.0001], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([0.7903], grad_fn=<SigmoidBackward>)
output is: tensor([1.3320e-32], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([1.5609e-24], grad_fn=<SigmoidBackward>)
output is: tensor([0.3709], grad_fn=<SigmoidBackward>)
output is: tensor([0.9958], grad_fn=<SigmoidBackward>)
output is: tensor([1.0026e-09], grad_fn=<SigmoidBackward>)
output is: tensor([0.0405], grad_fn=<SigmoidBackward>)
output is: tensor([0.0020], grad_fn=<SigmoidBackward>)
ou

output is: tensor([8.2565e-30], grad_fn=<SigmoidBackward>)
output is: tensor([6.0764e-11], grad_fn=<SigmoidBackward>)
output is: tensor([2.2539e-05], grad_fn=<SigmoidBackward>)
output is: tensor([3.5658e-23], grad_fn=<SigmoidBackward>)
output is: tensor([0.9997], grad_fn=<SigmoidBackward>)
output is: tensor([8.3498e-11], grad_fn=<SigmoidBackward>)
output is: tensor([8.2806e-31], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([1.0000], grad_fn=<SigmoidBackward>)
output is: tensor([0.4835], grad_fn=<SigmoidBackward>)
output is: tensor([9.4307e-08], grad_fn=<SigmoidBackward>)
output is: tensor([5.5443e-07], grad_fn=<SigmoidBackward>)
output is: tensor([0.0001], grad_fn=<SigmoidBackward>)
output is: tensor([0.0057], grad_fn=<SigmoidBackward>)
output is: tensor([8.7555e-05], grad_fn=<SigmoidBackward>)
output is: tensor([1.], grad_fn=<SigmoidBackward>)
output is: tensor([0.6484], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<Sigm

output is: tensor([0.3957], grad_fn=<SigmoidBackward>)
output is: tensor([3.0331e-38], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([8.4584e-30], grad_fn=<SigmoidBackward>)
output is: tensor([0.0291], grad_fn=<SigmoidBackward>)
output is: tensor([0.9965], grad_fn=<SigmoidBackward>)
output is: tensor([1.1947e-23], grad_fn=<SigmoidBackward>)
output is: tensor([0.9999], grad_fn=<SigmoidBackward>)
output is: tensor([2.6622e-05], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([6.1695e-17], grad_fn=<SigmoidBackward>)
output is: tensor([7.9625e-13], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([3.7545e-06], grad_fn=<SigmoidBackward>)
output is: tensor([2.8309e-30], grad_fn=<SigmoidBackward>)
output is: tensor([3.4532e-31], grad_fn=<SigmoidBackward>)
output is: tensor([2.8321e-21], grad_fn=<Sigm

output is: tensor([2.5325e-10], grad_fn=<SigmoidBackward>)
output is: tensor([0.4478], grad_fn=<SigmoidBackward>)
output is: tensor([0.0005], grad_fn=<SigmoidBackward>)
output is: tensor([1.6738e-20], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([0.0007], grad_fn=<SigmoidBackward>)
output is: tensor([0.4955], grad_fn=<SigmoidBackward>)
output is: tensor([0.0054], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([3.5074e-27], grad_fn=<SigmoidBackward>)
output is: tensor([1.], grad_fn=<SigmoidBackward>)
output is: tensor([0.7778], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([2.8747e-05], grad_fn=<SigmoidBackward>)
output is: tensor([7.6706e-06], grad_fn=<SigmoidBackward>)
output is: tensor([0.1271], grad_fn=<SigmoidBackward>)
output is: ten

output is: tensor([3.9800e-31], grad_fn=<SigmoidBackward>)
output is: tensor([1.1095e-37], grad_fn=<SigmoidBackward>)
output is: tensor([4.3650e-09], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([2.6667e-18], grad_fn=<SigmoidBackward>)
output is: tensor([7.7266e-05], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([5.8948e-30], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([0.0116], grad_fn=<SigmoidBackward>)
output is: tensor([1.1225e-06], grad_fn=<SigmoidBackward>)
output is: tensor([7.1300e-31], grad_fn=<SigmoidBackward>)
output is: tensor([0.0007], grad_fn=<SigmoidBackward>)
output is: tensor([1.], grad_fn=<SigmoidBackward>)
output is: tensor([2.2421e-37], grad_fn=<SigmoidBackward>)
output is: tensor([0.7799], grad_fn=<SigmoidBackward>

output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([3.8463e-06], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([2.3641e-06], grad_fn=<SigmoidBackward>)
output is: tensor([0.0021], grad_fn=<SigmoidBackward>)
output is: tensor([8.8051e-34], grad_fn=<SigmoidBackward>)
output is: tensor([0.9963], grad_fn=<SigmoidBackward>)
output is: tensor([1.5701e-35], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([2.2937e-20], grad_fn=<SigmoidBackward>)
output is: tensor([3.4543e-11], grad_fn=<SigmoidBackward>)
output is: tensor([4.5289e-06], grad_fn=<SigmoidBackward>)
output is: tensor([0.0019], grad_fn=<SigmoidBackward>)
output is: tensor([0.0056], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([1.3028e-20], grad_fn=<SigmoidBackward>)
output is: tensor([0.0010], grad_fn=<SigmoidBackward>)
output is: tensor([0.0004], grad_fn=<SigmoidBackw

output is: tensor([0.0517], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([3.2442e-33], grad_fn=<SigmoidBackward>)
output is: tensor([0.0066], grad_fn=<SigmoidBackward>)
output is: tensor([0.0217], grad_fn=<SigmoidBackward>)
output is: tensor([1.7579e-17], grad_fn=<SigmoidBackward>)
output is: tensor([4.8682e-29], grad_fn=<SigmoidBackward>)
output is: tensor([0.0008], grad_fn=<SigmoidBackward>)
output is: tensor([0.9997], grad_fn=<SigmoidBackward>)
output is: tensor([4.3021e-10], grad_fn=<SigmoidBackward>)
output is: tensor([2.2515e-23], grad_fn=<SigmoidBackward>)
output is: tensor([8.2641e-12], grad_fn=<SigmoidBackward>)
output is: tensor([0.9992], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([0.], grad_fn=<SigmoidBackward>)
output is: tensor([2.1843e-13], grad_fn=<SigmoidBackward>)
output is: tensor([0.0338], grad_fn=<SigmoidBackward>